<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Post-data-(raw-data)" data-toc-modified-id="Import-Post-data-(raw-data)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Post data (raw data)</a></span><ul class="toc-item"><li><span><a href="#Train-set-(raw-data)" data-toc-modified-id="Train-set-(raw-data)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Train set (raw data)</a></span></li><li><span><a href="#Val-set-(raw-data)" data-toc-modified-id="Val-set-(raw-data)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Val set (raw data)</a></span></li></ul></li><li><span><a href="#Import-Tag-data" data-toc-modified-id="Import-Tag-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Tag data</a></span><ul class="toc-item"><li><span><a href="#Train-set-(Tag-data)" data-toc-modified-id="Train-set-(Tag-data)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Train set (Tag data)</a></span></li><li><span><a href="#Val-set-(Tag-data)" data-toc-modified-id="Val-set-(Tag-data)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Val set (Tag data)</a></span></li></ul></li><li><span><a href="#Dataset-pipeline" data-toc-modified-id="Dataset-pipeline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataset pipeline</a></span></li><li><span><a href="#Train-model-:-universal-sentence-encoder-+-RN-dense" data-toc-modified-id="Train-model-:-universal-sentence-encoder-+-RN-dense-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train model : universal sentence encoder + RN dense</a></span></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate</a></span><ul class="toc-item"><li><span><a href="#Train-set" data-toc-modified-id="Train-set-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Train set</a></span></li><li><span><a href="#Val-set" data-toc-modified-id="Val-set-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Val set</a></span></li></ul></li></ul></div>

In [1]:
import os
import itertools
import re
import string
import pandas as pd
import glob
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, f1_score

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras.backend as K
print('Tensorflow version : {}'.format(tf.__version__))

Tensorflow version : 2.1.0


# Import Post data (raw data)

## Train set (raw data)

In [3]:
import pickle5

In [4]:
#read the pickle file
picklefile = open('train/X_train_filtre.pkl', 'rb')
#unpickle the dataframe
df_X_train = pickle5.load(picklefile)
#close file
picklefile.close()

#print the dataframe
df_X_train.shape

(103225, 4)

In [5]:
df_X_train.head(5)

,raw,Text,text_small,text_large
195918,how to store parts of formdata when theyre on ...,"[store, part, formdata, -PRON-, be, separate, ...","[store, part, separate, page, whenever, prepar...","[store, part, -PRON-, be, separate, page, when..."
495592,which tools should no php web developer live w...,"[tool, php, web, developer, live, without, lot...","[tool, php, web, developer, live, without, lot...","[tool, php, web, developer, live, without, lot..."
451531,long strings with newlines i have seen c# code...,"[long, string, newline, see, c, code, use, tel...","[long, string, newline, tell, compiler, string...","[long, string, newline, see, c, code, use, tel..."
542821,about character pointers in c consider this de...,"[character, pointers, c, consider, definitiona...","[character, pointers, consider, point, area, m...","[character, pointers, c, consider, see, point,..."
549984,how to install pythonrsvg without pythongnome2...,"[install, pythonrsvg, without, pythongnome, de...","[install, without, desktop, ubuntu, support, p...","[install, without, desktop, ubuntu, need, supp..."


In [6]:
# X dataframe to numpy
X_train = df_X_train.drop(columns=['Text', 'text_small', 'text_large']).values  # keep raw post
print('X shape : {}'.format(X_train.shape))

X shape : (103225, 1)


## Val set (raw data)

In [7]:
#read the pickle file
picklefile = open('val/X_val_filtre.pkl', 'rb')
#unpickle the dataframe
df_X_val = pickle5.load(picklefile)
#close file
picklefile.close()

#print the dataframe
df_X_val.shape

(34477, 2)

In [8]:
df_X_val.head(5)

,raw,Text
230075,what type of java cache should be used in case...,"[type, java, cache, use, case, data, change, f..."
295941,php how can i show n 0 t x0b r how can i show...,"[php, show, n, x, b, r, show, character, webpage]"
279171,saving hebrew utf8 to database with php when s...,"[saving, hebrew, utf, database, php, saving, h..."
201268,can nlog store the current thread user i am us...,"[nlog, store, current, thread, user, use, nlog..."
403510,how do i add a cgpoint to nsmutablearray i wan...,"[add, cgpoint, nsmutablearray, want, store, cg..."


In [9]:
# X dataframe to numpy
X_val = df_X_val.drop(columns=['Text']).values  # keep raw post
print('X shape : {}'.format(X_val.shape))

X shape : (34477, 1)


# Import Tag data

## Train set (Tag data)

In [10]:
#read the pickle file
picklefile = open('train/Y_train_filtre.pkl', 'rb')
#unpickle the dataframe
df_Y_train = pickle5.load(picklefile)
#close file
picklefile.close()

In [11]:
df_Y_train.head()

,Tags,tags_filtered
195918,"[html, database, forms, cookies]","[html, database, forms]"
495592,"[php, development-environment]",[php]
451531,"[c++, string]","[c++, string]"
542821,"[c, arrays, pointers]","[c, arrays]"
549984,"[python, librsvg, rsvg]",[python]


In [12]:
# binarize Y
mlb = MultiLabelBinarizer()
Y_train = mlb.fit_transform(df_Y_train['tags_filtered'])
print('Y shape : {}'.format(Y_train.shape))

Y shape : (103225, 100)


## Val set (Tag data)

In [13]:
#read the pickle file
picklefile = open('val/Y_val_filtre.pkl', 'rb')
#unpickle the dataframe
df_Y_val = pickle5.load(picklefile)
#close file
picklefile.close()

In [14]:
df_Y_val.head()

,Tags,tags_filtered
230075,"[java, performance, multithreading, caching, lru]","[caching, java, performance, multithreading]"
295941,"[php, hidden-characters]",[php]
279171,"[php, mysql, utf-8]","[mysql, php]"
201268,"[c#, .net, nlog]","[c#, .net]"
403510,"[iphone, objective-c, nsarray, cgpoint]","[iphone, objective-c]"


In [15]:
# binarize Y whith train binarizer
Y_val = mlb.fit_transform(df_Y_val['tags_filtered'])
print('Y shape : {}'.format(Y_val.shape))

Y shape : (34477, 100)


# Dataset pipeline

In [16]:
# nombre de tags
tags_count = Y_train.shape[1]
print('Nombre de tags : {}'.format(tags_count))

Nombre de tags : 100


In [17]:
# universal sentence encoder
encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [18]:
def get_dataset(X, Y, batch_size):
    def generator():
        example_count = X.shape[0]
        for i in range(example_count):
            x = tf.reshape((tf.convert_to_tensor(X[i], dtype=tf.string)), (1,))
            x = tf.reshape(encoder(x), (512,))  # encode post
            y = Y[i, :]
            yield x, y

    dataset = tf.data.Dataset.from_generator(generator,
                                             output_types=(
                                                 tf.float64, tf.float64),
                                             output_shapes=(tf.TensorShape([512, ]),
                                                            tf.TensorShape([Y.shape[1], ])))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    return dataset

In [19]:
batch_size = 256
train_dataset = get_dataset(X_train, Y_train, batch_size)
val_dataset = get_dataset(X_val, Y_val, batch_size)

In [20]:
for data in train_dataset.take(1):
    print(data)

(<tf.Tensor: shape=(256, 512), dtype=float64, numpy=
array([[-0.00398598, -0.06714278,  0.00383451, ...,  0.06920544,
         0.02128837, -0.04849395],
       [ 0.00999459, -0.05821686,  0.01879022, ..., -0.02598965,
        -0.05236601, -0.06074374],
       [ 0.05299769, -0.0424128 ,  0.05116303, ...,  0.07500345,
         0.01311447,  0.00184043],
       ...,
       [-0.04255071, -0.00389851, -0.01476747, ...,  0.06234134,
        -0.04690759, -0.06022419],
       [-0.06327029,  0.00036987,  0.06723199, ...,  0.05274028,
         0.04898303,  0.02141742],
       [-0.01893879, -0.00063862,  0.00702118, ..., -0.02978746,
        -0.03101772, -0.00472722]])>, <tf.Tensor: shape=(256, 100), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>)


# Train model : universal sentence encoder + RN dense

In [21]:
inputs = tf.keras.Input(shape=(512,))

x = tf.keras.layers.Dense(512, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(tags_count, activation='sigmoid')(x)
outputs = x

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='tags_model')

model.summary()

Model: "tags_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
Total params: 313,956
Trainable params: 313,956
Non-trainable params: 0
_________________________________________________________________


In [22]:
def f1_loss(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return 1 - K.mean(f1)

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              #             loss=[tf.keras.losses.binary_crossentropy],
              loss=[f1_loss],
              metrics=[tf.keras.metrics.Precision(thresholds=0.5), tf.keras.metrics.Recall(thresholds=0.5)])

In [24]:
model.fit(train_dataset, epochs=5, validation_data=val_dataset,
          validation_steps=None)

Epoch 1/5
404/404 [==============================] - 339s 840ms/step - loss: 0.7549 - precision: 0.1270 - recall: 0.5858 - val_loss: 0.6143 - val_precision: 0.4797 - val_recall: 0.6089
Epoch 2/5
404/404 [==============================] - 334s 827ms/step - loss: 0.5944 - precision: 0.4904 - recall: 0.6243 - val_loss: 0.5649 - val_precision: 0.5162 - val_recall: 0.6344
Epoch 3/5
404/404 [==============================] - 335s 828ms/step - loss: 0.5617 - precision: 0.5149 - recall: 0.6414 - val_loss: 0.5463 - val_precision: 0.5334 - val_recall: 0.6423
Epoch 4/5
404/404 [==============================] - 335s 828ms/step - loss: 0.5451 - precision: 0.5275 - recall: 0.6498 - val_loss: 0.5351 - val_precision: 0.5413 - val_recall: 0.6477
Epoch 5/5
404/404 [==============================] - 335s 830ms/step - loss: 0.5335 - precision: 0.5349 - recall: 0.6571 - val_loss: 0.5279 - val_precision: 0.5482 - val_recall: 0.6506


In [25]:
# save modele
model_config = model.get_config()
weights = model.get_weights()
# save JSON config to disk
json_config = model.to_json()
with open('PickleData/RN_usencoder2tags_model_config.json', 'w') as json_file:
    json_file.write(json_config)
# save weights to disk
model.save_weights('PickleData/RN_usencoder2tag_weights.h5')

In [26]:
# save tags
import pickle
tags = mlb.classes_
with open('PickleData/RN_usencoder2tags_tags', 'wb') as f:
    pickle.dump(tags, f, pickle.HIGHEST_PROTOCOL)

# Evaluate

## Train set 

In [27]:
# predict train
print('Nombre de tags dans le jeu de train :{}'.format(sum(sum(Y_train))))
y_pred = np.concatenate(np.array([model.predict(data[0]) for data in train_dataset]))
# seuillage
Y_pred_train = (y_pred > 0.5) * 1
print('Nombre de tags prédits :{}'.format(sum(sum(Y_pred_train))))
train_score = f1_score(Y_train, Y_pred_train, average='macro')  # seuillage

Nombre de tags dans le jeu de train :168815


C:\Users\Mini\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


Nombre de tags prédits :200765


## Val set 

In [28]:
# predict val
print('Nombre de tags dans le jeu de val :{}'.format(sum(sum(Y_val))))
y_pred = np.concatenate(np.array([model.predict(data[0]) for data in val_dataset]))
# seuillage
Y_pred_val = (y_pred > 0.5) * 1
print('Nombre de tags prédits :{}'.format(sum(sum(Y_pred_val))))

Nombre de tags dans le jeu de val :56448
Nombre de tags prédits :66995


C:\Users\Mini\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# save results
csv_path = 'results/univ_sent_encoder'
clf_name = 'univ_sent_encoder'
train_score_macro = f1_score(Y_train, Y_pred_train, average='macro')
val_score_macro = f1_score(Y_val, Y_pred_val, average='macro')
train_score_micro = f1_score(Y_train, Y_pred_train, average='micro')
val_score_micro = f1_score(Y_val, Y_pred_val, average='micro')
train_score_weighted = f1_score(Y_train, Y_pred_train, average='weighted')
val_score_weighted = f1_score(Y_val, Y_pred_val, average='weighted')
train_score_samples = f1_score(Y_train, Y_pred_train, average='samples')
val_score_samples = f1_score(Y_val, Y_pred_val, average='samples')
with open(csv_path, 'w') as file:
    file.write('{};{};{};{};{};{};{};{};{}'.format(clf_name,
                                                   train_score_macro,
                                                   val_score_macro,
                                                   train_score_micro,
                                                   val_score_micro,
                                                   train_score_weighted,
                                                   val_score_weighted,
                                                   train_score_samples,
                                                   val_score_samples))

In [30]:
print(classification_report(Y_val, Y_pred_val))

              precision    recall  f1-score   support

           0       0.30      0.59      0.40      2619
           1       0.56      0.62      0.59       303
           2       0.53      0.56      0.55       470
           3       0.58      0.58      0.58       333
           4       0.96      0.78      0.86       438
           5       0.62      0.67      0.64       218
           6       0.62      0.65      0.64       346
           7       0.37      0.64      0.47       294
           8       0.62      0.60      0.61      2253
           9       0.84      0.68      0.75       811
          10       0.47      0.72      0.56       144
          11       0.47      0.48      0.47       802
          12       0.47      0.68      0.55      5163
          13       0.74      0.63      0.68      1858
          14       0.56      0.70      0.62       151
          15       0.38      0.51      0.43       337
          16       0.22      0.61      0.33       331
          17       0.74    

C:\Users\Mini\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
